## 环境要求
硬件：Ascend 910

MindSpore：2.2.0

cann7.0rc1

## mindformers安装


In [3]:
#  下载链接
!git clone -b dev https://gitee.com/mindspore/mindformers.git


Cloning into 'mindformers'...
remote: Enumerating objects: 25530, done.
remote: Counting objects: 100% (4223/4223), done.
remote: Compressing objects: 100% (2231/2231), done.
remote: Total 25530 (delta 2993), reused 2863 (delta 1975), pack-reused 21307
Receiving objects: 100% (25530/25530), 39.66 MiB | 4.54 MiB/s, done.
Resolving deltas: 100% (18585/18585), done.


In [3]:
#进入mindformers/setup.py文件中修改版本为0.8.0,修改requirements.txt文件的mindpet==1.0.2,tokenizers==0.14.0
#开始安装
# %cd mindformers
!bash build.sh

---------------- MindFormers: build start ----------------
running bdist_wheel
running build
running build_py
creating build/lib/mindformers
copying mindformers/mindformer_book.py -> build/lib/mindformers
copying mindformers/version_control.py -> build/lib/mindformers
copying mindformers/__init__.py -> build/lib/mindformers
copying mindformers/auto_class.py -> build/lib/mindformers
creating build/lib/mindformers/dataset
copying mindformers/dataset/mask_language_model_dataset.py -> build/lib/mindformers/dataset
copying mindformers/dataset/img_cls_dataset.py -> build/lib/mindformers/dataset
copying mindformers/dataset/labels.py -> build/lib/mindformers/dataset
copying mindformers/dataset/contrastive_language_image_pretrain_dataset.py -> build/lib/mindformers/dataset
copying mindformers/dataset/text_classification_dataset.py -> build/lib/mindformers/dataset
copying mindformers/dataset/translation_dataset.py -> build/lib/mindformers/dataset
copying mindformers/dataset/reward_model_dataset.

In [8]:
%cd ../

/home/ma-user/work


## 获取MindFormers提供的已转换权重

In [9]:
!wget https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/llama/open_llama_7b.ckpt

--2024-01-23 16:46:45--  https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/llama/open_llama_7b.ckpt
Resolving proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)... 192.168.0.106
Connecting to proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)|192.168.0.106|:8083... connected.
Proxy request sent, awaiting response... 200 OK
Length: 13476850247 (13G) [binary/octet-stream]
Saving to: ‘open_llama_7b.ckpt’

open_llama_7b.ckpt  100%[===================>]  12.55G  83.9MB/s    in 2m 17s  

2024-01-23 16:49:03 (93.6 MB/s) - ‘open_llama_7b.ckpt’ saved [13476850247/13476850247]



In [11]:
# tokenizer文件
!wget https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/llama/tokenizer.model

--2024-01-23 16:49:39--  https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/llama/tokenizer.model
Resolving proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)... 192.168.0.106
Connecting to proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)|192.168.0.106|:8083... connected.
Proxy request sent, awaiting response... 200 OK
Length: 534194 (522K) [binary/octet-stream]
Saving to: ‘tokenizer.model’

tokenizer.model     100%[===================>] 521.67K  --.-KB/s    in 0.1s    

2024-01-23 16:49:39 (3.74 MB/s) - ‘tokenizer.model’ saved [534194/534194]



## 微调
数据集准备-微调
目前提供alpaca数据集的预处理脚本用于全参微调/lora微调任务。

数据集下载链接如下：
https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json

alpaca数据集原始格式样例：

```json
# alpaca examples:
    {
        "instruction": "Describe a time when you had to make a difficult decision.",
        "input": "",
        "output": "I had to make a difficult decision when I was working as a project manager at a construction company. I was in charge of a project that needed to be completed by a certain date in order to meet the client\u2019s expectations. However, due to unexpected delays, we were not able to meet the deadline and so I had to make a difficult decision. I decided to extend the deadline, but I had to stretch the team\u2019s resources even further and increase the budget. Although it was a risky decision, I ultimately decided to go ahead with it to ensure that the project was completed on time and that the client\u2019s expectations were met. The project was eventually successfully completed and this was seen as a testament to my leadership and decision-making abilities."
    },
    {
        "instruction": "Identify the odd one out.",
        "input": "Twitter, Instagram, Telegram",
        "output": "Telegram"
    },
```




In [14]:
#step 1. 执行alpaca_converter.py，使用fastchat工具添加prompts模板，将原始数据集转换为多轮对话格式。
# 脚本路径：tools/dataset_preprocess/llama/alpaca_converter.py
# 执行转换脚本
!python mindformers/mindformers/tools/dataset_preprocess/llama/alpaca_converter.py --data_path alpaca_data.json --output_path alpaca-data-conversation.json

In [3]:
%pip install fschat==0.2.13
%pip install tk

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
#step 2. 执行llama_preprocess.py，进行数据预处理、Mindrecord数据生成，将带有prompt模板的数据转换为mindrecord格式。

#注：由于此工具依赖fschat工具包解析prompt模板，请提前安装fschat >= 0.2.13 python = 3.9

# 脚本路径：tools/dataset_preprocess/llama/llama_preprocess.py
!python mindformers/mindformers/tools/dataset_preprocess/llama/llama_preprocess.py  --dataset_type qa --input_glob alpaca-data-conversation.json  --model_file tokenizer.model --seq_length 2048 --output_file alpaca-fastchat2048.mindrecord

Traceback (most recent call last):
  File "/home/ma-user/work/mindformers/mindformers/tools/dataset_preprocess/llama/llama_preprocess.py", line 25, in <module>
    from mindspore.mindrecord import FileWriter
ModuleNotFoundError: No module named 'mindspore'


## lora微调

step 1. 修改config/llama/run_llama_7b.yaml中训练数据集路径为微调数据集路径,并在input_columns中添加labels

``` yaml
train_dataset: &train_dataset
  data_loader:
    type: MindDataset
    dataset_dir: "/{path}/alpaca-fastchat2048.mindrecord"
    shuffle: True
  input_columns: ["input_ids", "labels"]
```

step 2. 启动lora微调任务。

注：llama_7b_lora模型支持单卡启动，需将配置文件中的use_parallel参数置为False。

In [5]:
# %cd mindformers/scripts
# 单卡启动
!bash run_standalone.sh ../configs/llama/run_llama_7b_lora.yaml 0 finetune

output_dir is /home/ma-user/work/mindformers/output
start training for device 0
log saved in /home/ma-user/work/mindformers/output/log/rank_0


## 评测
文本生成

step 1. 获取数据集

WikiText2数据集是从维基百科上经过验证的优质文章集中提取的超过1亿个token的集合。(https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip)

step 2. 处理数据成mindrecord格式

In [7]:
%pwd
%cd ../

/home/ma-user/work/mindformers


In [11]:
# 使用tools/dataset_preprocess/llama/llama_preprocess.py进行数据预处理+Mindrecord数据生成
!python /home/ma-user/work/mindformers/mindformers/tools/dataset_preprocess/llama/llama_preprocess.py \
--dataset_type wiki \
--input_glob  /home/ma-user/work/wikitext-2/wiki.valid.tokens \
--model_file /home/ma-user/work/tokenizer.model \
--seq_length 2047 \
--output_file /home/ma-user/work/wiki2048.mindrecord

#因为我这里已经安装过mindformers以及安装了一些冲突的依赖，所以我在启智社区重新创建了一个云脑任务，在云脑中，只安装mindformers，
#可以成功转换,重新创建一个云脑，依次运行以下代码，就不会依赖冲突。
#git clone -b dev https://gitee.com/mindspore/mindformers.git
# cd mindformers
# bash build.sh

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

step 3. 开启评测，指标为PPL

In [6]:
%pip install pyarrow

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 35.6 MB 69.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python /home/ma-user/work/mindformers/run_mindformer.py \
--config /home/ma-user/work/configs/llama/run_llama_7b.yaml \
--eval_dataset_dir /home/ma-user/work/wiki2048.mindrecord \
--run_mode eval \
--load_checkpoint llama_7b \
--epochs 1 \
--use_parallel False \
--device_id 0

# PerplexityMetric = {'PerplexityMetric': {'loss': 2.1142693907022476, 'PPL': 8.283531529594038}}

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 